# 08 - Evaluation on Test-Shift with Threshold-Only Adaptation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IvanNece/Detection-of-Anomalies-with-Localization/blob/main/notebooks/08_evaluation_shifted_threshold_only.ipynb)

**Phase 6.2: Threshold-Only Adaptation**

This notebook implements an ablation study to isolate the contribution of **threshold recalibration** from full model adaptation.

**Experimental Setup:**
- Models: Trained on Clean data (NO retraining)
- Thresholds: RE-calibrated on Val-shift (F1-optimal)
- Test: Evaluated on Test-shift

**Goal:**
Measure how much performance can be recovered through threshold adaptation alone, without model retraining.

**Metrics computed:**
- Image-level: AUROC, AUPRC, F1, Accuracy, Precision, Recall
- Pixel-level: Pixel AUROC, PRO (Per-Region Overlap)

## 1. Setup & Configuration

In [ ]:
# ============================================================
# SETUP - Mount Google Drive & Clone Repository
# ============================================================

from google.colab import drive
from pathlib import Path
import os
import sys

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Done!\n")

# Clone repository on main branch
print("Cloning repository (branch: main)...")
repo_dir = '/content/Detection-of-Anomalies-with-Localization'

# Remove if exists
if os.path.exists(repo_dir):
    print("Removing existing repository...")
    !rm -rf {repo_dir}

# Clone from main branch
!git clone https://github.com/IvanNece/Detection-of-Anomalies-with-Localization.git {repo_dir}
print("Done!\n")

# Setup paths
PROJECT_ROOT = Path(repo_dir)

# Dataset location
DATASET_PATH = Path('/content/drive/MyDrive/mvtec_shifted')

# Output directories on Drive
DRIVE_ROOT = Path('/content/drive/MyDrive/anomaly_detection_project')
PATCHCORE_MODELS_DIR = DRIVE_ROOT / '04_patchcore_clean_outputs'
PADIM_MODELS_DIR = DRIVE_ROOT / '05_padim_clean_outputs'

RESULTS_DIR = PROJECT_ROOT / 'outputs' / 'results'
THRESHOLDS_DIR = PROJECT_ROOT / 'outputs' / 'thresholds'
VIZ_DIR = PROJECT_ROOT / 'outputs' / 'visualizations' / 'shift_threshold_only'

RESULTS_DIR.mkdir(parents=True, exist_ok=True)
THRESHOLDS_DIR.mkdir(parents=True, exist_ok=True)
VIZ_DIR.mkdir(parents=True, exist_ok=True)

# Verify dataset exists
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset not found at {DATASET_PATH}\n"
        f"Please ensure mvtec_shifted folder is in your Google Drive root."
    )

# Add project root to Python path
sys.path.insert(0, str(PROJECT_ROOT))

print("\n" + "="*70)
print("SETUP COMPLETE")
print("="*70)
print(f"Project:   {PROJECT_ROOT}")
print(f"Dataset:   {DATASET_PATH}")
print(f"PatchCore: {PATCHCORE_MODELS_DIR}")
print(f"PaDiM:     {PADIM_MODELS_DIR}")
print(f"Results:   {RESULTS_DIR}")
print(f"Viz:       {VIZ_DIR}")
print("="*70)

Install both `faiss` and `anomalib` libraries, required for running PatchCore and PaDiM models respectively. **Must be done before any imports.**

In [ ]:
!pip install faiss-cpu --quiet
!pip install anomalib --quiet

Import necessary libraries and modules.

In [ ]:
# Standard imports
import json
import warnings
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# Project imports
from src.utils.reproducibility import set_seed
from src.utils.config import load_config
from src.utils.paths import ProjectPaths
from src.data.splitter import load_splits
from src.data.dataset import MVTecDataset
from src.data.transforms import get_clean_transforms
from src.models.patchcore import PatchCore
from src.models.padim_wrapper import PadimWrapper

# Metrics imports
from src.metrics import (
    calibrate_threshold,
    ThresholdCalibrator,
    compute_image_metrics,
    compute_pixel_metrics,
    compute_roc_curve,
    compute_pr_curve,
    compute_confusion_matrix,
    aggregate_metrics,
    aggregate_pixel_metrics
)

warnings.filterwarnings('ignore')

# Set random seed for reproducibility
set_seed(42)

# Load configuration
config = load_config(PROJECT_ROOT / 'configs' / 'experiment_config.yaml')
paths = ProjectPaths(PROJECT_ROOT)

# Classes to evaluate
CLASSES = config.dataset.classes  # ['hazelnut', 'carpet', 'zipper']

# Device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


def evaluation_collate(batch):
    """Custom collate function to handle None masks in batches."""
    batch = list(zip(*batch))
    images = torch.stack(batch[0])
    masks = batch[1] # Keep as tuple/list to handle None
    labels = torch.tensor(batch[2])
    paths = batch[3]
    return images, masks, labels, paths

print(f"Using device: {DEVICE}")
print(f"Classes: {CLASSES}")

## 2. Load Shifted Data Splits

In [ ]:
# Load shifted splits
SPLITS_PATH = paths.get_split_path('shifted')
splits = load_splits(SPLITS_PATH)

# Print split statistics
print("\nSplit Statistics:")
print("-" * 50)
for class_name in CLASSES:
    val_n = len(splits[class_name]['val']['images'])
    test_n = len(splits[class_name]['test']['images'])

    val_normal = sum(1 for l in splits[class_name]['val']['labels'] if l == 0)
    val_anom = sum(1 for l in splits[class_name]['val']['labels'] if l == 1)
    test_normal = sum(1 for l in splits[class_name]['test']['labels'] if l == 0)
    test_anom = sum(1 for l in splits[class_name]['test']['labels'] if l == 1)

    print(f"{class_name}:")
    print(f"  Val: {val_n} ({val_normal} normal, {val_anom} anomalous)")
    print(f"  Test: {test_n} ({test_normal} normal, {test_anom} anomalous)")

## 3. Threshold Calibration on Val-Shift

**CRITICAL STEP:** We recalibrate thresholds using predictions from **clean-trained models** on **Val-shift** data.

This isolates the effect of threshold adaptation from model adaptation.

In [ ]:
# Initialize threshold calibrators
patchcore_calibrator = ThresholdCalibrator('patchcore')
padim_calibrator = ThresholdCalibrator('padim')

# Get transforms
transform = get_clean_transforms(image_size=config.dataset.image_size)

# Store calibration data for visualization
calibration_data = {
    'patchcore': {},
    'padim': {}
}

print("=" * 60)
print("THRESHOLD CALIBRATION (F1-Optimal on Val-shift)")
print("=" * 60)

In [ ]:
# PATCHCORE: Calibrate thresholds on Val-shift
print("\n>>> PATCHCORE <<<\n")

for class_name in CLASSES:
    print(f"\n--- {class_name.upper()} ---")

    # Create validation dataset
    val_split = splits[class_name]['val']
    val_dataset = MVTecDataset.from_split(
        val_split,
        transform=transform,
        phase='val'
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        collate_fn=evaluation_collate
    )

    # Load model trained on CLEAN data
    model = PatchCore(
        backbone_layers=config.patchcore.layers,
        patch_size=config.patchcore.patch_size,
        coreset_ratio=config.patchcore.coreset_sampling_ratio,
        n_neighbors=config.patchcore.n_neighbors,
        device=DEVICE
    )
    model.load(PATCHCORE_MODELS_DIR, class_name, domain='clean')

    # Collect predictions on VAL-SHIFT
    val_scores = []
    val_labels = []

    with torch.no_grad():
        for images, masks, labels, paths in tqdm(val_loader, desc=f'Validating {class_name}'):
            images = images.to(DEVICE)
            scores, _ = model.predict(images, return_heatmaps=False)

            val_scores.extend(scores.tolist())
            val_labels.extend(labels.numpy().tolist())

    # Convert to arrays
    val_scores = np.array(val_scores)
    val_labels = np.array(val_labels)

    # Calibrate threshold on Val-shift
    threshold = patchcore_calibrator.calibrate(class_name, val_scores, val_labels)
    print(f"  Calibrated threshold: {threshold:.4f}")

    # Store for visualization
    calibration_data['patchcore'][class_name] = {
        'scores': val_scores,
        'labels': val_labels,
        'threshold': threshold
    }

# Save calibrated thresholds
patchcore_calibrator.save(THRESHOLDS_DIR / 'shift_threshold_only_patchcore.json')
print(f"\n[OK] Thresholds saved: shift_threshold_only_patchcore.json")

In [ ]:
# PADIM: Calibrate thresholds on Val-shift
print("\n>>> PADIM <<<\n")

for class_name in CLASSES:
    print(f"\n--- {class_name.upper()} ---")

    # Create validation dataset
    val_split = splits[class_name]['val']
    val_dataset = MVTecDataset.from_split(
        val_split,
        transform=transform,
        phase='val'
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        collate_fn=evaluation_collate
    )

    # Load model trained on CLEAN data
    model = PadimWrapper(
        backbone=config.padim.backbone,
        layers=config.padim.layers,
        n_features=config.padim.n_features,
        image_size=config.dataset.image_size,
        device=DEVICE
    )
    model.load(PADIM_MODELS_DIR / f'padim_{class_name}_clean.pt')

    # Collect predictions on VAL-SHIFT
    val_scores = []
    val_labels = []

    with torch.no_grad():
        for images, masks, labels, paths in tqdm(val_loader, desc=f'Validating {class_name}'):
            images = images.to(DEVICE)
            scores, _ = model.predict(images, return_heatmaps=False)

            val_scores.extend(scores.tolist())
            val_labels.extend(labels.numpy().tolist())

    # Convert to arrays
    val_scores = np.array(val_scores)
    val_labels = np.array(val_labels)

    # Calibrate threshold on Val-shift
    threshold = padim_calibrator.calibrate(class_name, val_scores, val_labels)
    print(f"  Calibrated threshold: {threshold:.4f}")

    # Store for visualization
    calibration_data['padim'][class_name] = {
        'scores': val_scores,
        'labels': val_labels,
        'threshold': threshold
    }

# Save calibrated thresholds
padim_calibrator.save(THRESHOLDS_DIR / 'shift_threshold_only_padim.json')
print(f"\n[OK] Thresholds saved: shift_threshold_only_padim.json")

## 4. Visualization: Score Distributions & Thresholds

Visualizing how anomaly scores are distributed on Val-shifted data and where the new thresholds fall.

In [ ]:
# Visualization: Score distributions and thresholds
fig, axes = plt.subplots(len(CLASSES), 2, figsize=(14, 4*len(CLASSES)))

for i, class_name in enumerate(CLASSES):
    # PatchCore
    ax1 = axes[i, 0]
    pc_data = calibration_data['patchcore'][class_name]
    normal_scores = pc_data['scores'][pc_data['labels'] == 0]
    anomalous_scores = pc_data['scores'][pc_data['labels'] == 1]

    ax1.hist(normal_scores, bins=30, alpha=0.6, label='Normal', color='blue')
    ax1.hist(anomalous_scores, bins=30, alpha=0.6, label='Anomalous', color='red')
    ax1.axvline(pc_data['threshold'], color='green', linestyle='--', linewidth=2,
                label=f'Threshold: {pc_data["threshold"]:.2f}')
    ax1.set_title(f'PatchCore - {class_name}', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Anomaly Score')
    ax1.set_ylabel('Count')
    ax1.legend()

    # PaDiM
    ax2 = axes[i, 1]
    pd_data = calibration_data['padim'][class_name]
    normal_scores = pd_data['scores'][pd_data['labels'] == 0]
    anomalous_scores = pd_data['scores'][pd_data['labels'] == 1]

    ax2.hist(normal_scores, bins=30, alpha=0.6, label='Normal', color='blue')
    ax2.hist(anomalous_scores, bins=30, alpha=0.6, label='Anomalous', color='red')
    ax2.axvline(pd_data['threshold'], color='green', linestyle='--', linewidth=2,
                label=f'Threshold: {pd_data["threshold"]:.2f}')
    ax2.set_title(f'PaDiM - {class_name}', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Anomaly Score')
    ax2.set_ylabel('Count')
    ax2.legend()

plt.suptitle('Score Distributions & Calibrated Thresholds (Val-shift)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(VIZ_DIR / 'score_distributions_shift_threshold.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Evaluation on Test-Shift with Recalibrated Thresholds

Now we evaluate on the test set using the **recalibrated thresholds from Val-shift**.

In [ ]:
# Load recalibrated thresholds from saved results
thresholds = {
    'patchcore': patchcore_calibrator.thresholds,
    'padim': padim_calibrator.thresholds
}

# Storage for results
all_results = {
    'patchcore': {},
    'padim': {}
}

In [ ]:
# PATCHCORE: Evaluate on Test-shift
print("\n" + "="*60)
print("TEST-SHIFT EVALUATION (PATCHCORE)")
print("="*60)

for class_name in CLASSES:
    print(f"\n--- {class_name.upper()} ---")

    # Create test dataset
    test_split = splits[class_name]['test']
    test_dataset = MVTecDataset.from_split(
        test_split,
        transform=transform,
        phase='test'
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        collate_fn=evaluation_collate
    )

    # Load model trained on CLEAN data
    model = PatchCore(
        backbone_layers=config.patchcore.layers,
        patch_size=config.patchcore.patch_size,
        coreset_ratio=config.patchcore.coreset_sampling_ratio,
        n_neighbors=config.patchcore.n_neighbors,
        device=DEVICE
    )
    model.load(PATCHCORE_MODELS_DIR, class_name, domain='clean')

    # Collect predictions
    all_scores = []
    all_labels = []
    all_heatmaps = []
    all_masks = []

    with torch.no_grad():
        for images, masks, labels, paths in tqdm(test_loader, desc=f'Testing {class_name}'):
            images = images.to(DEVICE)
            scores, heatmaps = model.predict(images, return_heatmaps=True)

            all_scores.extend(scores.tolist())
            all_labels.extend(labels.numpy().tolist())

            for mask in masks:
                if mask is not None:
                    all_masks.append(mask.numpy().squeeze())
                else:
                    all_masks.append(None)

            all_heatmaps.extend([h for h in heatmaps])

    # Convert to arrays
    test_scores = np.array(all_scores)
    test_labels = np.array(all_labels)

    # Get recalibrated threshold
    threshold = thresholds['patchcore'][class_name]

    # Compute image-level metrics
    image_metrics = compute_image_metrics(test_labels, test_scores, threshold=threshold)

    # Compute pixel-level metrics
    pixel_metrics = compute_pixel_metrics(all_masks, all_heatmaps, compute_pro_metric=True)

    # Store results
    all_results['patchcore'][class_name] = {
        'threshold': threshold,
        'image_level': image_metrics,
        'pixel_level': pixel_metrics,
        'test_scores': test_scores.tolist(),
        'test_labels': test_labels.tolist()
    }

    # Print results
    print(f"  Threshold (recalibrated on val-shift): {threshold:.4f}")
    print(f"  AUROC: {image_metrics['auroc']:.4f}")
    print(f"  AUPRC: {image_metrics['auprc']:.4f}")
    print(f"  F1: {image_metrics['f1']:.4f}")
    print(f"  Accuracy: {image_metrics.get('accuracy', 'N/A'):.4f}")
    print(f"  Pixel AUROC: {pixel_metrics.get('pixel_auroc', 'N/A')}")
    print(f"  PRO: {pixel_metrics.get('pro', 'N/A')}")

In [ ]:
# PADIM: Evaluate on Test-shift
print("\n" + "="*60)
print("TEST-SHIFT EVALUATION (PADIM)")
print("="*60)

for class_name in CLASSES:
    print(f"\n--- {class_name.upper()} ---")

    # Create test dataset
    test_split = splits[class_name]['test']
    test_dataset = MVTecDataset.from_split(
        test_split,
        transform=transform,
        phase='test'
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        collate_fn=evaluation_collate
    )

    # Load model trained on CLEAN data
    model = PadimWrapper(
        backbone=config.padim.backbone,
        layers=config.padim.layers,
        n_features=config.padim.n_features,
        image_size=config.dataset.image_size,
        device=DEVICE
    )
    model.load(PADIM_MODELS_DIR / f'padim_{class_name}_clean.pt')

    # Collect predictions
    all_scores = []
    all_labels = []
    all_heatmaps = []
    all_masks = []

    with torch.no_grad():
        for images, masks, labels, paths in tqdm(test_loader, desc=f'Testing {class_name}'):
            images = images.to(DEVICE)
            scores, heatmaps = model.predict(images, return_heatmaps=True)

            all_scores.extend(scores.tolist())
            all_labels.extend(labels.numpy().tolist())

            for mask in masks:
                if mask is not None:
                    all_masks.append(mask.numpy().squeeze())
                else:
                    all_masks.append(None)

            all_heatmaps.extend([h for h in heatmaps])

    # Convert to arrays
    test_scores = np.array(all_scores)
    test_labels = np.array(all_labels)

    # Get recalibrated threshold
    threshold = thresholds['padim'][class_name]

    # Compute image-level metrics
    image_metrics = compute_image_metrics(test_labels, test_scores, threshold=threshold)

    # Compute pixel-level metrics
    pixel_metrics = compute_pixel_metrics(all_masks, all_heatmaps, compute_pro_metric=True)

    # Store results
    all_results['padim'][class_name] = {
        'threshold': threshold,
        'image_level': image_metrics,
        'pixel_level': pixel_metrics,
        'test_scores': test_scores.tolist(),
        'test_labels': test_labels.tolist()
    }

    # Print results
    print(f"  Threshold (recalibrated on val-shift): {threshold:.4f}")
    print(f"  AUROC: {image_metrics['auroc']:.4f}")
    print(f"  AUPRC: {image_metrics['auprc']:.4f}")
    print(f"  F1: {image_metrics['f1']:.4f}")
    print(f"  Accuracy: {image_metrics.get('accuracy', 'N/A'):.4f}")
    print(f"  Pixel AUROC: {pixel_metrics.get('pixel_auroc', 'N/A')}")
    print(f"  PRO: {pixel_metrics.get('pro', 'N/A')}")

## 6. Save Results & Copy to Drive

In [ ]:
# Helper to convert results for JSON serialization
def convert_for_json(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_for_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_for_json(i) for i in obj]
    else:
        return obj

# Save full results JSON
json_path = RESULTS_DIR / 'shifted_threshold_only_results.json'
with open(json_path, 'w') as f:
    json.dump(convert_for_json(all_results), f, indent=2)
print(f"[OK] Results saved: {json_path.name}")

# Create summary CSV
import pandas as pd
summary_rows = []

for method, method_results in all_results.items():
    for class_name, res in method_results.items():
        row = {
            'Method': method.upper(),
            'Class': class_name,
            'Threshold': res['threshold'],
            'AUROC': res['image_level']['auroc'],
            'AUPRC': res['image_level']['auprc'],
            'F1': res['image_level']['f1'],
            'Accuracy': res['image_level']['accuracy'],
            'Pixel AUROC': res['pixel_level']['pixel_auroc'],
            'PRO': res['pixel_level']['pro']
        }
        summary_rows.append(row)

# Create DataFrame
df = pd.DataFrame(summary_rows)

# Add Macro Average
for method in ['PATCHCORE', 'PADIM']:
    method_df = df[df['Method'] == method]
    if not method_df.empty:
        macro_avg = method_df.mean(numeric_only=True)
        macro_row = macro_avg.to_dict()
        macro_row['Method'] = method
        macro_row['Class'] = 'Macro_average'
        summary_rows.append(macro_row)

final_df = pd.DataFrame(summary_rows)
csv_path = RESULTS_DIR / 'shifted_threshold_only_results_summary.csv'
final_df.to_csv(csv_path, index=False)
print(f"[OK] Summary saved: {csv_path.name}")

print("\n" + "="*60)
print("SUMMARY RESULTS (Threshold-Only Adaptation)")
print("="*60)
print(final_df[['Method', 'Class', 'AUROC', 'F1', 'Pixel AUROC', 'PRO']].to_string(index=False))

In [ ]:
# Copy Results to Drive for persistence

# Create destination folders
THRESHOLD_ONLY_OUTPUTS = DRIVE_ROOT / '08_threshold_only_outputs'
THRESHOLD_ONLY_OUTPUTS.mkdir(parents=True, exist_ok=True)

import shutil

generated_files = [
    THRESHOLDS_DIR / 'shift_threshold_only_patchcore.json',
    THRESHOLDS_DIR / 'shift_threshold_only_padim.json',
    RESULTS_DIR / 'shifted_threshold_only_results.json',
    RESULTS_DIR / 'shifted_threshold_only_results_summary.csv',
    VIZ_DIR / 'score_distributions_shift_threshold.png'
]

print("\nCopying files to Drive...")
for src in generated_files:
    if src.exists():
        dst = THRESHOLD_ONLY_OUTPUTS / src.name
        shutil.copy(src, dst)
        print(f"  [COPIED] {src.name}")
    else:
        print(f"  [MISSING] {src.name}")

print(f"\nAll results saved to: {THRESHOLD_ONLY_OUTPUTS}")